# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 9:17AM,254889,12,HH8742.2,Hush Hush,Released
1,Jan 12 2023 9:11AM,254888,10,CTF0011589,"Citieffe, Inc.",Released
2,Jan 12 2023 9:11AM,254888,10,CTF0011590,"Citieffe, Inc.",Released
3,Jan 12 2023 9:07AM,254885,10,0086295510,ISDIN Corporation,Released
4,Jan 12 2023 9:07AM,254885,10,0086294480,ISDIN Corporation,Released
5,Jan 12 2023 9:07AM,254885,10,0086294493,ISDIN Corporation,Released
6,Jan 12 2023 9:07AM,254885,10,0086294492,ISDIN Corporation,Released
7,Jan 12 2023 9:07AM,254885,10,0086294497,ISDIN Corporation,Released
8,Jan 12 2023 9:07AM,254885,10,0086294498,ISDIN Corporation,Released
9,Jan 12 2023 9:07AM,254885,10,0086294299,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,254883,Released,33
45,254885,Released,24
46,254886,Released,4
47,254888,Released,2
48,254889,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254883,NaN,NaN,33.0
254885,NaN,NaN,24.0
254886,NaN,NaN,4.0
254888,NaN,NaN,2.0
254889,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254753,0.0,0.0,2.0
254770,7.0,3.0,2.0
254777,0.0,2.0,2.0
254799,0.0,0.0,1.0
254801,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254753,0,0,2
254770,7,3,2
254777,0,2,2
254799,0,0,1
254801,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254753,0,0,2
1,254770,7,3,2
2,254777,0,2,2
3,254799,0,0,1
4,254801,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254753,,,2
1,254770,7,3,2
2,254777,,2,2
3,254799,,,1
4,254801,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 9:17AM,254889,12,Hush Hush
1,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc."
3,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation
27,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation
60,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation
93,Jan 12 2023 9:01AM,254886,10,Emerginnova
97,Jan 12 2023 8:57AM,254882,16,Sartorius Stedim Filters Inc.
98,Jan 12 2023 8:48AM,254879,16,"Emersa Waterbox, LLC"
99,Jan 12 2023 8:44AM,254878,10,ISDIN Corporation
104,Jan 12 2023 8:42AM,254877,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 9:17AM,254889,12,Hush Hush,,,1
1,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",,,2
2,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,,,24
3,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,,,33
4,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,,,33
5,Jan 12 2023 9:01AM,254886,10,Emerginnova,,,4
6,Jan 12 2023 8:57AM,254882,16,Sartorius Stedim Filters Inc.,,,1
7,Jan 12 2023 8:48AM,254879,16,"Emersa Waterbox, LLC",,,1
8,Jan 12 2023 8:44AM,254878,10,ISDIN Corporation,,,5
9,Jan 12 2023 8:42AM,254877,12,Hush Hush,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:17AM,254889,12,Hush Hush,1,,
1,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",2,,
2,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,24,,
3,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,33,,
4,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,33,,
5,Jan 12 2023 9:01AM,254886,10,Emerginnova,4,,
6,Jan 12 2023 8:57AM,254882,16,Sartorius Stedim Filters Inc.,1,,
7,Jan 12 2023 8:48AM,254879,16,"Emersa Waterbox, LLC",1,,
8,Jan 12 2023 8:44AM,254878,10,ISDIN Corporation,5,,
9,Jan 12 2023 8:42AM,254877,12,Hush Hush,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:17AM,254889,12,Hush Hush,1,,
1,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",2,,
2,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,24,,
3,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,33,,
4,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,33,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:17AM,254889,12,Hush Hush,1.0,NaN,NaN
1,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",2.0,NaN,NaN
2,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,24.0,NaN,NaN
3,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,33.0,NaN,NaN
4,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,33.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 9:17AM,254889,12,Hush Hush,1.0,0.0,0.0
1,Jan 12 2023 9:11AM,254888,10,"Citieffe, Inc.",2.0,0.0,0.0
2,Jan 12 2023 9:07AM,254885,10,ISDIN Corporation,24.0,0.0,0.0
3,Jan 12 2023 9:07AM,254883,10,ISDIN Corporation,33.0,0.0,0.0
4,Jan 12 2023 9:07AM,254881,10,ISDIN Corporation,33.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3567787,113.0,7.0,8.0
12,509766,1.0,1.0,0.0
15,764545,115.0,0.0,33.0
16,764635,3.0,0.0,0.0
19,509731,63.0,0.0,0.0
21,4586865,18.0,0.0,0.0
22,254871,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3567787,113.0,7.0,8.0
1,12,509766,1.0,1.0,0.0
2,15,764545,115.0,0.0,33.0
3,16,764635,3.0,0.0,0.0
4,19,509731,63.0,0.0,0.0
5,21,4586865,18.0,0.0,0.0
6,22,254871,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,113.0,7.0,8.0
1,12,1.0,1.0,0.0
2,15,115.0,0.0,33.0
3,16,3.0,0.0,0.0
4,19,63.0,0.0,0.0
5,21,18.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,113.0
1,12,Released,1.0
2,15,Released,115.0
3,16,Released,3.0
4,19,Released,63.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,8.0,0.0,33.0,0.0,0.0,0.0,0.0
Executing,7.0,1.0,0.0,0.0,0.0,0.0,1.0
Released,113.0,1.0,115.0,3.0,63.0,18.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,8.0,0.0,33.0,0.0,0.0,0.0,0.0
1,Executing,7.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Released,113.0,1.0,115.0,3.0,63.0,18.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,8.0,0.0,33.0,0.0,0.0,0.0,0.0
1,Executing,7.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Released,113.0,1.0,115.0,3.0,63.0,18.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()